# Ollama API를 이용한 문서 질의응답 시스템

이 노트북은 doc 폴더의 마크다운 파일들을 읽어서 Ollama API를 통해 질의응답을 수행합니다.

In [1]:
import os
import glob
import ollama
import json
from pathlib import Path
import pandas as pd
from IPython.display import display, Markdown

print("라이브러리 로딩 완료")

라이브러리 로딩 완료


## 1. 마크다운 파일 로딩 함수

In [2]:
def load_markdown_files(doc_folder='doc'):
    """
    doc 폴더의 모든 마크다운 파일을 읽어서 딕셔너리로 반환
    """
    documents = {}
    doc_path = Path(doc_folder)
    
    if not doc_path.exists():
        print(f"'{doc_folder}' 폴더가 존재하지 않습니다.")
        return documents
    
    md_files = list(doc_path.glob('*.md'))
    
    if not md_files:
        print(f"'{doc_folder}' 폴더에 마크다운 파일이 없습니다.")
        return documents
    
    for md_file in md_files:
        try:
            with open(md_file, 'r', encoding='utf-8') as f:
                content = f.read()
                documents[md_file.name] = {
                    'path': str(md_file),
                    'content': content,
                    'size': len(content)
                }
                print(f"✅ {md_file.name} 로딩 완료 ({len(content):,} 문자)")
        except Exception as e:
            print(f"❌ {md_file.name} 로딩 실패: {e}")
    
    return documents

# 문서 로딩
documents = load_markdown_files()
print(f"\n총 {len(documents)}개의 문서가 로딩되었습니다.")

✅ guide.md 로딩 완료 (17,863 문자)

총 1개의 문서가 로딩되었습니다.


## 2. 로딩된 문서 정보 확인

In [3]:
if documents:
    doc_info = []
    for filename, doc_data in documents.items():
        doc_info.append({
            '파일명': filename,
            '경로': doc_data['path'],
            '크기(문자)': f"{doc_data['size']:,}",
            '내용 미리보기': doc_data['content'][:100] + '...' if len(doc_data['content']) > 100 else doc_data['content']
        })
    
    df_docs = pd.DataFrame(doc_info)
    display(df_docs)
else:
    print("로딩된 문서가 없습니다.")

,파일명,경로,크기(문자),내용 미리보기
0,guide.md,doc/guide.md,"17,863",# bizMOB Client TypeScript Library Guide\n\nbi...


## 3. Ollama API 연결 및 모델 확인

In [4]:
def check_ollama_connection():
    """
    Ollama 서비스 연결 및 사용 가능한 모델 확인
    """
    try:
        # 사용 가능한 모델 목록 가져오기
        models = ollama.list()
        print("🔗 Ollama 연결 성공!")
        print(f"사용 가능한 모델 수: {len(models['models'])}")
        
        model_list = []
        for model in models['models']:
            # 모델 정보 안전하게 추출
            model_name = model.get('model', model.get('name', 'Unknown'))
            model_size = model.get('size', 0)
            model_modified = model.get('modified_at', 'N/A')
            
            model_list.append({
                '모델명': model_name,
                '크기': f"{model_size / (1024**3):.1f}GB" if model_size > 0 else 'N/A',
                '수정일': str(model_modified)[:10] if model_modified != 'N/A' else 'N/A'
            })
        
        if model_list:
            df_models = pd.DataFrame(model_list)
            display(df_models)
            # 첫 번째 모델을 기본으로 사용
            first_model = models['models'][0]
            return first_model.get('model', first_model.get('name', 'llama3.1'))
        else:
            print("사용 가능한 모델이 없습니다. 먼저 모델을 다운로드해주세요.")
            print("예: ollama pull llama2")
            return None
            
    except Exception as e:
        print(f"❌ Ollama 연결 실패: {e}")
        print("Ollama가 실행 중인지 확인해주세요: ollama serve")
        return None

# Ollama 연결 확인
default_model = check_ollama_connection()

🔗 Ollama 연결 성공!
사용 가능한 모델 수: 7


,모델명,크기,수정일
0,gpt-oss:latest,12.8GB,2025-08-20
1,qwen3:latest,4.9GB,2025-07-28
2,joonoh/HyperCLOVAX-SEED-Text-Instruct-1.5B:latest,3.0GB,2025-07-28
3,exaone3.5:latest,4.4GB,2025-07-28
4,llama3.1:latest,4.6GB,2025-07-15
5,deepseek-r1:latest,4.9GB,2025-07-15
6,llama3.1:8b,4.6GB,2025-07-15


## 4. 문서 기반 질의응답 함수

In [5]:
def query_documents(question, model_name=None, include_context=True):
    """
    로딩된 문서를 컨텍스트로 사용하여 질문에 답변
    """
    if not documents:
        return "로딩된 문서가 없습니다. 먼저 문서를 로딩해주세요."
    
    if not model_name:
        model_name = default_model
        
    if not model_name:
        return "사용 가능한 Ollama 모델이 없습니다."
    
    # 모든 문서 내용을 컨텍스트로 결합
    context = ""
    for filename, doc_data in documents.items():
        context += f"\n\n=== {filename} ===\n{doc_data['content']}"
    
    # 프롬프트 구성
    if include_context:
        prompt = f"""다음 문서들을 기반으로 질문에 답변해주세요:

문서 내용:
{context}

질문: {question}

답변은 문서의 내용을 기반으로 정확하고 자세하게 해주세요. 문서에 없는 내용은 추측하지 마세요."""
    else:
        prompt = question
    
    try:
        print(f"🤖 {model_name} 모델로 질의 중...")
        
        # Ollama API 호출
        response = ollama.chat(model=model_name, messages=[
            {
                'role': 'user',
                'content': prompt
            }
        ])
        
        return response['message']['content']
        
    except Exception as e:
        return f"❌ 질의 실패: {e}"

print("질의응답 함수 정의 완료")

질의응답 함수 정의 완료


## 5. 대화형 질의응답 인터페이스

In [6]:
def interactive_qa():
    """
    대화형 질의응답 인터페이스
    """
    print("=" * 60)
    print("📚 문서 기반 질의응답 시스템")
    print("=" * 60)
    print("사용법:")
    print("- 질문을 입력하면 로딩된 문서를 기반으로 답변합니다")
    print("- 'quit' 또는 'exit'를 입력하면 종료됩니다")
    print("- '!docs'를 입력하면 로딩된 문서 목록을 확인할 수 있습니다")
    print("-" * 60)
    
    while True:
        question = input("\n❓ 질문을 입력하세요: ").strip()
        
        if question.lower() in ['quit', 'exit', '종료']:
            print("👋 질의응답을 종료합니다.")
            break
        
        if question == '!docs':
            print("\n📁 로딩된 문서 목록:")
            for i, (filename, doc_data) in enumerate(documents.items(), 1):
                print(f"{i}. {filename} ({doc_data['size']:,} 문자)")
            continue
        
        if not question:
            print("질문을 입력해주세요.")
            continue
        
        print("\n" + "="*50)
        answer = query_documents(question)
        print("\n💬 답변:")
        print(answer)
        print("="*50)

# 대화형 인터페이스 시작 (필요시 실행)
# interactive_qa()

## 6. 예시 질의응답

In [7]:
# 예시 질문들
sample_questions = [
    "문서에 어떤 내용이 포함되어 있나요?",
    "주요 키워드를 요약해주세요",
    "문서의 구조를 설명해주세요"
]

print("📝 예시 질의응답:")
print("=" * 60)

for i, question in enumerate(sample_questions, 1):
    print(f"\n{i}. 질문: {question}")
    print("-" * 40)
    
    answer = query_documents(question)
    print(f"답변: {answer}")
    print("=" * 60)

📝 예시 질의응답:

1. 질문: 문서에 어떤 내용이 포함되어 있나요?
----------------------------------------
🤖 gpt-oss:latest 모델로 질의 중...
답변: **문서에 포함된 주요 내용은 다음과 같습니다.**  

| 항목 | 내용 요약 |
|------|-----------|
| **문서 개요** | bizMOB Client Library 사용 가이드: 네이티브 기능 활용법, 설치, 설정, 주요 API 목록, 인증/암호화 흐름, 에러 코드 참조, 사용 예제. |
| **소개** | bizMOB Client Library가 제공하는 핵심 기능과 사용 목적에 대한 설명. |
| **시작하기** | **① Vite 기반 설치** (`pnpm i @bizmob-client`), **② vite.config.js** 설정 예시, **③ vite.config.ts** 예시, **④ Vue 프로젝트에서 사용 예시** (`main.ts`에서 `createApp`). |
| **API 레퍼런스** | 주요 모듈 및 클래스별 상세 API 정의와 사용법. |
| **Network** | - 네트워크 요청 기본 구조 (`_sTrcode`, `_oBody`, `_bMock`, `_bProgressEnable`).<br>- `request()`, `requestLogin()`, `requestCommon()`, `requestCommonBatch()` 메서드 예시와 사용법. |
| **Storage** | - 로컬 스토리지(`window.localStorage`)와 비휘발성 저장(`SecureStorage`) 기반 `set`, `get`, `delete`, `clear` 메서드.<br>- `setUserItem`, `setUserInfo`, `setSystemInfo` 예시. |
| **File** | - 파일 다운로드(`download`), 업로드(`upload`), 압축(`compress`), 압축 해제(`uncompress`), 이

## 7. 커스텀 질의

In [8]:
# 여기에 직접 질문을 입력하여 테스트할 수 있습니다
custom_question = "문서의 주요 내용을 3줄로 요약해주세요"

if custom_question.strip():
    print(f"❓ 질문: {custom_question}")
    print("="*50)
    
    answer = query_documents(custom_question)
    print(f"\n💬 답변:")
    display(Markdown(answer))
else:
    print("위의 custom_question 변수에 질문을 입력하고 셀을 실행하세요.")

❓ 질문: 문서의 주요 내용을 3줄로 요약해주세요
🤖 gpt-oss:latest 모델로 질의 중...

💬 답변:


BizMOB Client Library는 네이티브 기능을 활용한 통신, 파일 관리, 암호화, JWT 토큰 및 다국어 지원 등을 TypeScript 기반으로 제공하며, Mock 데이터 옵션과 이벤트 시스템, 설정 관리 기능을 갖추고 있다.  
라이브러리는 비동기 Promise 기반 API를 통해 인증, 파일 업로드, 데이터 요청 등 핵심 서비스를 지원하며, 각 API는 타입 정의와 에러 코드 처리 로직을 제공한다.  
문서에서는 모킹(JSON) 데이터 사용법, 에러 처리 가이드, IDE 자동완성 활용 등을 통해 개발자가 안정적이고 효율적으로 bizMOB 기능을 프로젝트에 통합할 수 있도록 설명한다.

## 8. 고급 기능 - 문서별 개별 질의

In [22]:
def query_specific_document(filename, question, model_name=None):
    """
    특정 문서에 대해서만 질의
    """
    if filename not in documents:
        return f"'{filename}' 문서를 찾을 수 없습니다."
    
    if not model_name:
        model_name = default_model
    
    if not model_name:
        return "사용 가능한 Ollama 모델이 없습니다."
    
    context = documents[filename]['content']
    
    prompt = f"""다음 문서를 기반으로 질문에 답변해주세요:

문서: {filename}
내용:
{context}

질문: {question}

답변은 이 문서의 내용만을 기반으로 해주세요."""
    
    try:
        print(f"🤖 {model_name} 모델로 '{filename}' 문서 질의 중...")
        
        response = ollama.chat(model=model_name, messages=[
            {
                'role': 'user',
                'content': prompt
            }
        ])
        
        return response['message']['content']
        
    except Exception as e:
        return f"❌ 질의 실패: {e}"

# 특정 문서 질의 예시
if documents:
    first_doc = list(documents.keys())[0]
    print(f"📄 '{first_doc}' 문서에 대한 질의 예시:")
    
    specific_answer = query_specific_document(first_doc, "이 문서의 핵심 내용이 무엇인가요?")
    print(f"\n답변: {specific_answer}")
else:
    print("질의할 문서가 없습니다.")

📄 'guide.md' 문서에 대한 질의 예시:
🤖 gpt-oss:latest 모델로 'guide.md' 문서 질의 중...

답변: **핵심 내용**  
본 문서는 **Vue 3 프로젝트에서 bizMOB Client Library**를 활용하기 위한 전체 가이드입니다.  
- **라이브러리 개요**: bizMOB 클라이언트와 Vue 3 간 연동 방식, 설정 파일(`vue.config.js`)과 환경 변수(`VUE_APP_*`)에 대한 설명이 포함됩니다.  
- **주요 API 모듈**: `Network`, `File`, `System`, `Event`, `Config`, `Logger` 등 네이티브 기능을 다루는 클래스와 각 클래스가 제공하는 메서드(예: `requestLogin`, `upload`, `callGallery`, `resizeImage`, `setEvent`, `clearEvent`, `set`, `get`)를 단계별로 소개합니다.  
- **인증 및 암호화**: JWT 토큰(`BzToken`)과 암호화 통신(`BzCrypto`)의 초기화, 갱신, 만료 확인 및 재발급 흐름을 설명하고, 관련 에러 코드(`ERR000`, `BM4002TKER1002`, `EAH000`, `EAH001`)에 따른 처리 방안을 제시합니다.  
- **다국어 처리**: `BzLocale`를 이용한 언어 초기화, 변경, 조회 방법을 안내합니다.  
- **네트워크 요청**: `Network.request()`와 `Network.requestLogin()` 같은 비동기 요청 패턴과 Mock 데이터 사용 방법을 예시와 함께 제공합니다.  
- **파일 및 시스템 기능**: 파일 업로드/다운로드, 압축, 리사이즈, 카메라·갤러리 호출, 프로그레스 표시 등 `File`과 `System` 클래스의 사용법을 단계별로 다룹니다.  
- **에러 코드와 예외 처리**: JWT, 암호화, 인증 관련 주요 에러 코드와 대응 방법, Promise 기반 API에서의 try‑catch 활용법

In [22]:
def query_specific_document(filename, question, model_name=None):
    """
    특정 문서에 대해서만 질의
    """
    if filename not in documents:
        return f"'{filename}' 문서를 찾을 수 없습니다."
    
    if not model_name:
        model_name = default_model
    
    if not model_name:
        return "사용 가능한 Ollama 모델이 없습니다."
    
    context = documents[filename]['content']
    
    prompt = f"""다음 문서를 기반으로 질문에 답변해주세요:

문서: {filename}
내용:
{context}

질문: {question}

답변은 이 문서의 내용만을 기반으로 해주세요."""
    
    try:
        print(f"🤖 {model_name} 모델로 '{filename}' 문서 질의 중...")
        
        response = ollama.chat(model=model_name, messages=[
            {
                'role': 'user',
                'content': prompt
            }
        ])
        
        return response['message']['content']
        
    except Exception as e:
        return f"❌ 질의 실패: {e}"

# 특정 문서 질의 예시
if documents:
    first_doc = list(documents.keys())[0]
    print(f"📄 '{first_doc}' 문서에 대한 질의 예시:")
    
    specific_answer = query_specific_document(first_doc, "이 문서의 핵심 내용이 무엇인가요?")
    print(f"\n답변: {specific_answer}")
else:
    print("질의할 문서가 없습니다.")

📄 'guide.md' 문서에 대한 질의 예시:
🤖 gpt-oss:latest 모델로 'guide.md' 문서 질의 중...

답변: **핵심 내용**  
본 문서는 **Vue 3 프로젝트에서 bizMOB Client Library**를 활용하기 위한 전체 가이드입니다.  
- **라이브러리 개요**: bizMOB 클라이언트와 Vue 3 간 연동 방식, 설정 파일(`vue.config.js`)과 환경 변수(`VUE_APP_*`)에 대한 설명이 포함됩니다.  
- **주요 API 모듈**: `Network`, `File`, `System`, `Event`, `Config`, `Logger` 등 네이티브 기능을 다루는 클래스와 각 클래스가 제공하는 메서드(예: `requestLogin`, `upload`, `callGallery`, `resizeImage`, `setEvent`, `clearEvent`, `set`, `get`)를 단계별로 소개합니다.  
- **인증 및 암호화**: JWT 토큰(`BzToken`)과 암호화 통신(`BzCrypto`)의 초기화, 갱신, 만료 확인 및 재발급 흐름을 설명하고, 관련 에러 코드(`ERR000`, `BM4002TKER1002`, `EAH000`, `EAH001`)에 따른 처리 방안을 제시합니다.  
- **다국어 처리**: `BzLocale`를 이용한 언어 초기화, 변경, 조회 방법을 안내합니다.  
- **네트워크 요청**: `Network.request()`와 `Network.requestLogin()` 같은 비동기 요청 패턴과 Mock 데이터 사용 방법을 예시와 함께 제공합니다.  
- **파일 및 시스템 기능**: 파일 업로드/다운로드, 압축, 리사이즈, 카메라·갤러리 호출, 프로그레스 표시 등 `File`과 `System` 클래스의 사용법을 단계별로 다룹니다.  
- **에러 코드와 예외 처리**: JWT, 암호화, 인증 관련 주요 에러 코드와 대응 방법, Promise 기반 API에서의 try‑catch 활용법

## 🚀 바이브 코딩 전용 함수 (Vue.js + TypeScript)

Vue.js와 TypeScript를 사용한 bizMOB 기반 하이브리드 앱 개발을 위한 전용 코딩 도구입니다.

In [9]:
def generate_vibe_code(requirement, component_type='component', model_name=None):
    """
    바이브 개발을 위한 Vue.js + TypeScript 코드 생성
    
    Args:
        requirement (str): 개발 요구사항 (예: "파일 업로드 기능이 있는 컴포넌트를 만들어줘")
        component_type (str): 'component', 'page', 'service', 'util' 중 선택
        model_name (str): 사용할 Ollama 모델 (기본값: default_model)
    
    Returns:
        str: 생성된 Vue.js + TypeScript 코드
    """
    if not documents:
        return "❌ bizMOB 문서가 로딩되지 않았습니다. 먼저 문서를 로딩해주세요."
    
    if not model_name:
        model_name = default_model
        
    if not model_name:
        return "❌ 사용 가능한 Ollama 모델이 없습니다."
    
    # bizMOB 가이드 문서 컨텍스트
    biz_mob_context = ""
    for filename, doc_data in documents.items():
        if 'guide' in filename.lower() or 'bizmob' in filename.lower():
            biz_mob_context += f"\\n\\n=== {filename} ===\\n{doc_data['content']}"
    
    # 컴포넌트 타입별 템플릿
    component_templates = {
        'component': """Vue.js 컴포넌트 (.vue 파일) 형태로 작성해주세요:
- <template> 섹션: HTML 템플릿
- <script setup lang="ts"> 섹션: TypeScript 로직
- <style scoped> 섹션: CSS 스타일링
- Composition API 사용
- TypeScript 타입 정의 포함""",
        
        'page': """Vue.js 페이지 컴포넌트 (.vue 파일) 형태로 작성해주세요:
- <template> 섹션: 페이지 레이아웃
- <script setup lang="ts"> 섹션: 페이지 로직
- <style scoped> 섹션: 페이지 스타일
- Vue Router 연동 고려
- 페이지 생명주기 활용""",
        
        'service': """TypeScript 서비스 클래스 (.ts 파일) 형태로 작성해주세요:
- 클래스 기반 서비스 구조
- bizMOB API 호출 로직
- 에러 처리 및 타입 안전성
- async/await 패턴 사용""",
        
        'util': """TypeScript 유틸리티 함수 (.ts 파일) 형태로 작성해주세요:
- 순수 함수 구조
- 타입 정의 포함
- 재사용 가능한 로직
- JSDoc 주석 추가"""
    }
    
    template = component_templates.get(component_type, component_templates['component'])
    
    # 프롬프트 구성
    prompt = f"""당신은 Vue.js와 TypeScript를 사용하여 bizMOB 기반 하이브리드 앱을 개발하는 전문가입니다.

다음 bizMOB 라이브러리 문서를 참고하여 코드를 생성해주세요:
{biz_mob_context}

**개발 요구사항:**
{requirement}

**코드 생성 규칙:**
{template}

**bizMOB 사용 가이드라인:**
1. bizMOB API 호출시 적절한 import 구문 사용
2. 에러 처리를 위한 try-catch 구문 포함
3. TypeScript 타입 안전성 확보
4. Vue.js Composition API 패턴 준수
5. 적절한 생명주기 메서드 활용
6. bizMOB 특화 기능 (인증, 암호화, 파일처리 등) 고려

**결과 형식:**
- 완전한 실행 가능한 코드 제공
- 주요 기능에 대한 간단한 설명 포함
- 필요한 import 구문 모두 포함
- 에러 처리 및 예외 상황 고려

코드를 생성해주세요:"""

    try:
        print(f"🔧 {model_name} 모델로 바이브 코드 생성 중...")
        print(f"📋 요구사항: {requirement}")
        print(f"🏗️  타입: {component_type}")
        
        # Ollama API 호출
        response = ollama.chat(model=model_name, messages=[
            {
                'role': 'user',
                'content': prompt
            }
        ])
        
        return response['message']['content']
        
    except Exception as e:
        return f"❌ 코드 생성 실패: {e}"

def generate_vibe_feature(feature_name, apis_needed=None, model_name=None):
    """
    특정 기능을 위한 완전한 Vue.js + TypeScript 기능 생성
    
    Args:
        feature_name (str): 기능명 (예: "사진 촬영 및 업로드")
        apis_needed (list): 필요한 bizMOB API들 (예: ['System.callCamera', 'File.upload'])
        model_name (str): 사용할 Ollama 모델
    
    Returns:
        dict: 컴포넌트, 서비스, 타입 등을 포함한 완전한 기능
    """
    if not documents:
        return "❌ bizMOB 문서가 로딩되지 않았습니다."
    
    if not model_name:
        model_name = default_model
        
    if not model_name:
        return "❌ 사용 가능한 Ollama 모델이 없습니다."
    
    # 관련 API 문서 필터링
    relevant_apis = ""
    if apis_needed:
        for filename, doc_data in documents.items():
            content = doc_data['content']
            for api in apis_needed:
                if api in content:
                    # API 관련 섹션 추출
                    lines = content.split('\\n')
                    for i, line in enumerate(lines):
                        if api in line:
                            # API 섹션 추출 (앞뒤 20줄)
                            start = max(0, i-10)
                            end = min(len(lines), i+30)
                            section = '\\n'.join(lines[start:end])
                            relevant_apis += f"\\n\\n=== {api} API ===\\n{section}"
                            break
    
    prompt = f"""당신은 Vue.js와 TypeScript를 사용하여 bizMOB 기반 하이브리드 앱을 개발하는 전문가입니다.

다음 기능을 완전히 구현해주세요: **{feature_name}**

**관련 bizMOB API 문서:**
{relevant_apis if relevant_apis else "전체 문서 참조"}

**생성할 파일들:**
1. **Vue 컴포넌트** (.vue): 사용자 인터페이스
2. **TypeScript 서비스** (.ts): bizMOB API 호출 로직
3. **TypeScript 타입** (.ts): 타입 정의
4. **사용법 가이드**: 구현된 기능의 사용 방법

**구현 요구사항:**
- Vue.js Composition API 사용
- TypeScript 타입 안전성 확보
- bizMOB API 적절한 활용
- 에러 처리 및 로딩 상태 관리
- 사용자 친화적 UI/UX
- 모바일 최적화 고려

각 파일을 구분하여 완전한 코드로 생성해주세요."""

    try:
        print(f"🏗️  {model_name} 모델로 '{feature_name}' 기능 생성 중...")
        if apis_needed:
            print(f"📡 필요한 API: {', '.join(apis_needed)}")
        
        response = ollama.chat(model=model_name, messages=[
            {
                'role': 'user',
                'content': prompt
            }
        ])
        
        return response['message']['content']
        
    except Exception as e:
        return f"❌ 기능 생성 실패: {e}"

def optimize_vibe_code(code, optimization_type='performance', model_name=None):
    """
    생성된 바이브 코드를 최적화
    
    Args:
        code (str): 최적화할 코드
        optimization_type (str): 'performance', 'security', 'maintainability'
        model_name (str): 사용할 Ollama 모델
    
    Returns:
        str: 최적화된 코드와 개선 사항 설명
    """
    if not model_name:
        model_name = default_model
        
    if not model_name:
        return "❌ 사용 가능한 Ollama 모델이 없습니다."
    
    optimization_guidelines = {
        'performance': """성능 최적화 관점에서 다음을 고려해주세요:
- Vue.js 반응성 최적화 (ref vs reactive 적절한 사용)
- 컴포넌트 지연 로딩 (lazy loading)
- 메모리 누수 방지
- 불필요한 리렌더링 방지
- bizMOB API 호출 최적화""",
        
        'security': """보안 최적화 관점에서 다음을 고려해주세요:
- 입력값 검증 및 새니타이징
- XSS, CSRF 방어
- 민감한 데이터 처리 방법
- bizMOB 인증/암호화 적절한 사용
- 에러 정보 노출 방지""",
        
        'maintainability': """유지보수성 최적화 관점에서 다음을 고려해주세요:
- 코드 구조 개선
- 재사용 가능한 컴포넌트 분리
- 타입 정의 강화
- 주석 및 문서화
- 테스트 가능한 구조"""
    }
    
    guideline = optimization_guidelines.get(optimization_type, optimization_guidelines['performance'])
    
    prompt = f"""다음 Vue.js + TypeScript + bizMOB 코드를 최적화해주세요.

**원본 코드:**
```
{code}
```

**최적화 유형:** {optimization_type}

**최적화 가이드라인:**
{guideline}

**결과 형식:**
1. **최적화된 코드**: 개선된 완전한 코드
2. **주요 개선 사항**: 어떤 부분이 어떻게 개선되었는지 설명
3. **추가 권장사항**: 더 나은 개선을 위한 제안

최적화를 수행해주세요:"""

    try:
        print(f"⚡ {model_name} 모델로 코드 최적화 중...")
        print(f"🎯 최적화 유형: {optimization_type}")
        
        response = ollama.chat(model=model_name, messages=[
            {
                'role': 'user',
                'content': prompt
            }
        ])
        
        return response['message']['content']
        
    except Exception as e:
        return f"❌ 코드 최적화 실패: {e}"

print("🚀 바이브 코딩 함수들이 정의되었습니다!")

🚀 바이브 코딩 함수들이 정의되었습니다!


## 🎯 바이브 코딩 실전 예시

다양한 bizMOB 기반 Vue.js + TypeScript 개발 예시들입니다.

In [10]:
# 바이브 코딩 예시 실행

# 예시 1: 파일 업로드 컴포넌트 생성
print("📁 예시 1: 파일 업로드 컴포넌트 생성")
print("=" * 50)

upload_component = generate_vibe_code(
    requirement="사용자가 사진을 선택하고 서버에 업로드할 수 있는 Vue 컴포넌트를 만들어줘. 업로드 진행 상태도 표시해야 해.",
    component_type="component"
)

print(f"생성된 코드:\\n{upload_component[:500]}...")
print("\\n" + "=" * 50)

# 예시 2: 카메라 촬영 기능 생성
print("📷 예시 2: 카메라 촬영 및 업로드 기능")
print("=" * 50)

camera_feature = generate_vibe_feature(
    feature_name="카메라 촬영 및 파일 업로드",
    apis_needed=["System.callCamera", "File.upload", "File.resizeImage"]
)

print(f"생성된 기능:\\n{camera_feature[:500]}...")
print("\\n" + "=" * 50)

📁 예시 1: 파일 업로드 컴포넌트 생성
🔧 gpt-oss:latest 모델로 바이브 코드 생성 중...
📋 요구사항: 사용자가 사진을 선택하고 서버에 업로드할 수 있는 Vue 컴포넌트를 만들어줘. 업로드 진행 상태도 표시해야 해.
🏗️  타입: component
생성된 코드:\n**ImageUploader.vue** – 사용자에게 사진을 선택하도록 하고, 선택한 사진을 한 번에 리사이즈한 뒤 서버에 업로드합니다.  
업로드가 진행되는 동안 `진행률`을 표시하고, 완료되면 업로드된 파일 URL을 화면에 표시합니다.  

```vue
<template>
  <div class="image-uploader">
    <button @click="selectAndUpload" :disabled="isUploading" class="btn">
      사진 선택 및 업로드
    </button>

    <!-- 업로드 진행중일 때 -->
    <div v-if="isUploading" class="progress-wrapper">
      <div class="progress-bar">
        <div class="progress" :style="{ width: progress + '%' }"></div>
      </div>
      <span...
\n==================================================
📷 예시 2: 카메라 촬영 및 업로드 기능
🏗️  gpt-oss:latest 모델로 '카메라 촬영 및 파일 업로드' 기능 생성 중...
📡 필요한 API: System.callCamera, File.upload, File.resizeImage
생성된 기능:\n## 📁 File Overview

| # | File | Purpose |
|---|------|---------|
| 1 | **UploadImage.vue** | Vue component – UI for selecting & uploading an

## 📚 바이브 코딩 사용 가이드

### 🔧 기본 사용법

#### 1. 간단한 컴포넌트 생성
```python
# Vue 컴포넌트 생성
code = generate_vibe_code(
    requirement="사용자 프로필을 표시하는 카드 컴포넌트",
    component_type="component"
)
```

#### 2. 완전한 기능 생성
```python
# 전체 기능 구현 (컴포넌트 + 서비스 + 타입)
feature = generate_vibe_feature(
    feature_name="QR코드 스캔 및 결과 처리",
    apis_needed=["Window.openCodeReader", "Storage.set"]
)
```

#### 3. 코드 최적화
```python
# 성능 최적화
optimized = optimize_vibe_code(
    code="your_vue_component_code",
    optimization_type="performance"
)
```

### 🎯 컴포넌트 타입 가이드

- **component**: 재사용 가능한 Vue 컴포넌트
- **page**: Vue Router용 페이지 컴포넌트
- **service**: bizMOB API 호출 서비스 클래스
- **util**: 유틸리티 함수 및 헬퍼

### 📱 주요 bizMOB 기능별 예시

#### 파일 관리
- "갤러리에서 이미지 선택 후 리사이즈하는 컴포넌트"
- "파일 다운로드 진행률을 표시하는 컴포넌트"
- "카메라로 사진 촬영하고 압축하는 기능"

#### 데이터 통신
- "서버 API 호출하는 서비스 클래스"
- "JWT 토큰 자동 갱신 기능"
- "암호화 통신 처리 서비스"

#### UI/UX
- "네이티브 스타일 로딩 인디케이터"
- "모바일 최적화 폼 컴포넌트"
- "스와이프 제스처 지원 이미지 뷰어"

### ⚠️ 주의사항
1. bizMOB API는 실제 디바이스/시뮬레이터에서만 동작합니다
2. 웹 환경에서는 Mock 데이터 사용을 고려하세요
3. TypeScript 타입 검사를 활용하여 런타임 에러를 방지하세요

## 🔍 코드 검증 및 포맷팅

In [ ]:
def validate_vibe_code(code, model_name=None):
    """
    생성된 바이브 코드의 품질과 표준 준수 여부를 검증
    
    Args:
        code (str): 검증할 코드
        model_name (str): 사용할 Ollama 모델
    
    Returns:
        dict: 검증 결과 및 개선 제안
    """
    if not model_name:
        model_name = default_model
        
    if not model_name:
        return {"error": "❌ 사용 가능한 Ollama 모델이 없습니다."}
    
    prompt = f"""다음 Vue.js + TypeScript + bizMOB 코드를 검증하고 품질 평가를 해주세요:

**코드:**
```
{code}
```

**검증 항목:**
1. **구문 정확성**: TypeScript/Vue.js 구문 오류 여부
2. **bizMOB API 사용**: API 사용법의 정확성
3. **타입 안전성**: TypeScript 타입 정의의 적절성
4. **Vue.js 베스트 프랙티스**: Composition API, 반응성 등
5. **에러 처리**: 예외 상황 처리의 적절성
6. **보안**: 잠재적 보안 취약점
7. **성능**: 성능상 이슈 가능성
8. **코드 품질**: 가독성, 유지보수성

**결과 형식:**
- 전체 점수: X/10
- 항목별 평가 (상/중/하)
- 발견된 문제점 목록
- 개선 제안
- 긍정적인 부분

검증을 수행해주세요:"""

    try:
        print(f"🔍 {model_name} 모델로 코드 검증 중...")
        
        response = ollama.chat(model=model_name, messages=[
            {
                'role': 'user',
                'content': prompt
            }
        ])
        
        return {
            "success": True,
            "validation_result": response['message']['content']
        }
        
    except Exception as e:
        return {"error": f"❌ 코드 검증 실패: {e}"}

def format_vibe_code(code, style='standard', model_name=None):
    """
    바이브 코드를 표준 스타일로 포맷팅
    
    Args:
        code (str): 포맷팅할 코드
        style (str): 'standard', 'compact', 'detailed' 중 선택
        model_name (str): 사용할 Ollama 모델
    
    Returns:
        str: 포맷팅된 코드
    """
    if not model_name:
        model_name = default_model
        
    if not model_name:
        return "❌ 사용 가능한 Ollama 모델이 없습니다."
    
    style_guidelines = {
        'standard': """표준 Vue.js + TypeScript 스타일 가이드를 적용해주세요:
- 2스페이스 들여쓰기
- 세미콜론 사용
- 단일 따옴표 사용
- 컴포넌트명은 PascalCase
- 변수명은 camelCase
- 적절한 줄바꿈과 공백""",
        
        'compact': """간결한 스타일로 포맷팅해주세요:
- 최소한의 공백 사용
- 한 줄에 여러 선언 허용
- 짧은 함수는 한 줄로 작성
- 불필요한 주석 제거""",
        
        'detailed': """상세한 스타일로 포맷팅해주세요:
- 충분한 공백과 줄바꿈
- 상세한 주석 추가
- 각 블록 간 여백
- 타입 정의 상세화"""
    }
    
    guideline = style_guidelines.get(style, style_guidelines['standard'])
    
    prompt = f"""다음 Vue.js + TypeScript + bizMOB 코드를 포맷팅해주세요:

**원본 코드:**
```
{code}
```

**포맷팅 스타일:** {style}

**포맷팅 가이드라인:**
{guideline}

**추가 요구사항:**
- 문법 오류가 있다면 수정
- 일관된 스타일 적용
- 가독성 개선
- Vue.js/TypeScript 컨벤션 준수

포맷팅된 코드만 반환해주세요:"""

    try:
        print(f"✨ {model_name} 모델로 코드 포맷팅 중...")
        print(f"🎨 스타일: {style}")
        
        response = ollama.chat(model=model_name, messages=[
            {
                'role': 'user',
                'content': prompt
            }
        ])
        
        return response['message']['content']
        
    except Exception as e:
        return f"❌ 코드 포맷팅 실패: {e}"

def explain_vibe_code(code, level='intermediate', model_name=None):
    """
    바이브 코드에 대한 상세한 설명 생성
    
    Args:
        code (str): 설명할 코드
        level (str): 'beginner', 'intermediate', 'advanced'
        model_name (str): 사용할 Ollama 모델
    
    Returns:
        str: 코드 설명
    """
    if not model_name:
        model_name = default_model
        
    if not model_name:
        return "❌ 사용 가능한 Ollama 모델이 없습니다."
    
    level_descriptions = {
        'beginner': """초보자를 위한 쉬운 설명:
- 각 줄의 기본적인 의미
- Vue.js와 TypeScript의 기본 개념
- bizMOB API의 역할
- 전체적인 동작 흐름""",
        
        'intermediate': """중급자를 위한 설명:
- 코드 구조와 설계 패턴
- Vue.js Composition API 활용
- TypeScript 타입 시스템
- bizMOB API 통합 방법
- 에러 처리 전략""",
        
        'advanced': """고급자를 위한 심화 설명:
- 아키텍처 설계 원칙
- 성능 최적화 기법
- 메모리 관리 전략
- 보안 고려사항
- 확장성 및 유지보수성"""
    }
    
    description_level = level_descriptions.get(level, level_descriptions['intermediate'])
    
    prompt = f"""다음 Vue.js + TypeScript + bizMOB 코드를 자세히 설명해주세요:

**코드:**
```
{code}
```

**설명 수준:** {level}

**설명 요구사항:**
{description_level}

**설명 구조:**
1. 전체 개요
2. 주요 구성 요소
3. 코드 흐름 분석
4. 사용된 기술과 패턴
5. 주의사항 및 팁

상세한 설명을 제공해주세요:"""

    try:
        print(f"📖 {model_name} 모델로 코드 설명 생성 중...")
        print(f"🎯 설명 수준: {level}")
        
        response = ollama.chat(model=model_name, messages=[
            {
                'role': 'user',
                'content': prompt
            }
        ])
        
        return response['message']['content']
        
    except Exception as e:
        return f"❌ 코드 설명 생성 실패: {e}"

print("🔍 코드 검증 및 포맷팅 함수들이 정의되었습니다!")

## 🎉 바이브 코딩 완료!

축하합니다! 바이브 코딩을 위한 모든 도구가 준비되었습니다.

### 🚀 사용 가능한 함수들

#### 📝 코드 생성
- `generate_vibe_code()` - Vue.js + TypeScript 컴포넌트/서비스 생성
- `generate_vibe_feature()` - 완전한 기능 구현 (컴포넌트 + 서비스 + 타입)

#### ⚡ 코드 최적화
- `optimize_vibe_code()` - 성능/보안/유지보수성 최적화

#### 🔍 코드 품질 관리  
- `validate_vibe_code()` - 코드 품질 검증 및 평가
- `format_vibe_code()` - 표준 스타일 포맷팅
- `explain_vibe_code()` - 코드 상세 설명

### 💡 빠른 시작

```python
# 1. 파일 업로드 컴포넌트 생성
upload_code = generate_vibe_code(
    "드래그 앤 드롭으로 파일을 업로드하는 Vue 컴포넌트",
    "component"
)

# 2. 카메라 촬영 기능 구현
camera_feature = generate_vibe_feature(
    "카메라로 사진을 촬영하고 서버에 업로드하는 기능",
    ["System.callCamera", "File.upload"]
)

# 3. 코드 검증
validation = validate_vibe_code(upload_code)
```

이제 bizMOB 문서를 기반으로 한 Vue.js + TypeScript 개발이 가능합니다! 🎯

# 10. 유틸리티 함수들

In [ ]:
def reload_documents():
    """
    문서 다시 로딩
    """
    global documents
    documents = load_markdown_files()
    print(f"문서 다시 로딩 완료: {len(documents)}개")

def show_document_stats():
    """
    문서 통계 표시
    """
    if not documents:
        print("로딩된 문서가 없습니다.")
        return
    
    total_chars = sum(doc_data['size'] for doc_data in documents.values())
    
    print("📊 문서 통계:")
    print(f"- 총 문서 수: {len(documents)}개")
    print(f"- 총 문자 수: {total_chars:,}개")
    print(f"- 평균 문서 크기: {total_chars//len(documents):,}문자")
    
    print("\n📁 개별 문서 크기:")
    for filename, doc_data in sorted(documents.items(), key=lambda x: x[1]['size'], reverse=True):
        print(f"  {filename}: {doc_data['size']:,}문자")

def export_qa_session(questions_answers, filename="qa_session.json"):
    """
    질의응답 세션을 JSON 파일로 저장
    """
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(questions_answers, f, ensure_ascii=False, indent=2)
        print(f"✅ 질의응답 세션이 '{filename}'에 저장되었습니다.")
    except Exception as e:
        print(f"❌ 저장 실패: {e}")

# 통계 표시
show_document_stats()

## 10. 사용 가이드

### 기본 사용법:
1. **문서 로딩**: `documents = load_markdown_files()` - doc 폴더의 모든 .md 파일을 로딩
2. **질의**: `answer = query_documents("질문")` - 로딩된 모든 문서를 기반으로 답변
3. **특정 문서 질의**: `answer = query_specific_document("파일명.md", "질문")` - 특정 문서만 사용

### 고급 기능:
- **대화형 인터페이스**: `interactive_qa()` 실행
- **문서 통계**: `show_document_stats()` 실행
- **문서 재로딩**: `reload_documents()` 실행

### 주의사항:
- Ollama가 실행 중이어야 합니다 (`ollama serve`)
- 최소 하나의 모델이 설치되어 있어야 합니다 (`ollama pull llama2`)
- 큰 문서의 경우 응답 시간이 오래 걸릴 수 있습니다
